In [1]:
import pandas as pd



In [2]:
import numpy as np


class University:
    def __init__(self, code, name, availabilities: np.ndarray):
        self.code = code
        self.name = name
        self.availabilities = availabilities

    def is_available(self, day, time):
        return self.availabilities[day, time]

    def get_schedule(self, day):
        return self.availabilities[day]

    def get_all_schedules(self):
        return self.availabilities

    def __str__(self):
        return f'{self.code} - {self.name}'

    def __repr__(self):
        return f'{self.code} - {self.name}'

    def __eq__(self, other):
        return self.code == other.code

    def __hash__(self):
        return hash(self.code)




In [7]:
# Create a list of University objects
import json

universities = []
with open('./data/allAvailabilities.json') as f:
    data = json.load(f)
    for university_code in data:
        code = university_code
        name = data[university_code]['university']

        availabilities = np.array([data[university_code]['isAvailable'][day] for day in ['M', 'T', 'W', 'Th', 'F']])
        universities.append(University(code, name, availabilities))

universities

[M1 - University of Toronto,
 M2 - Toronto Metropolitan University,
 M3 - University of Michigan,
 M4 - École Polytechnique de Montréal,
 M5 - University of Guelph,
 M6 - University of Calgary,
 M7 - University of Prince Edward Island,
 M8 - University of Manitoba,
 M9 - National Research Council,
 M10 - Royal Military College of Canada,
 M11 - Dalhousie University,
 M12 - University of Windsor,
 M13 - Queen’s University,
 M14 - The University of Western Ontario,
 M15 - École Polytechnique de Montréal,
 M16 - Université du Québec en Abitibi-Témiscamingue,
 M17 - Université du Québec à Trois Rivieres,
 M18 - FPrimeC Solutions Inc.,
 M19 - Institut national de la recherche scientifique,
 M20 - Université du Québec à Chicoutimi,
 M21 - Université Catholique de Louvain,
 M22 - University of Northern British Columbia,
 M23 - University of Saskatchewan,
 M24 - University of Alberta,
 M25 - University of Waterloo,
 M26 - University of Alberta,
 M27 - Brock University,
 M28 - University of Cyp

In [8]:
def indices_to_list(indices, length):
    return [1 if i in indices else 0 for i in range(length+1)][1:]

In [9]:


maxMeetings = pd.read_csv('./data/maxMeetings.csv')
maxMeetings['Time'] = maxMeetings['Time'].apply(lambda x: x - 24)
maxMeetings['Max Meetings'] = maxMeetings['Max Meetings'].astype(int)
maxMeetings

,Day,Time,Max Meetings
0,0,0,4
1,0,1,4
2,0,2,4
3,0,3,4
4,0,4,1
...,...,...,...
165,4,29,4
166,4,30,4
167,4,31,4
168,4,32,4


In [16]:
import itertools
import numpy as np


# Assuming 40 participants, 5 days, and 34 time slots
availabilities = np.zeros((40, 5, 34), dtype=int)  # Initialize with integer type

# Loop through each university and assign its schedule to the matrix
for i in range(40):
    schedule = universities[i].get_all_schedules()
    # Reshape the schedule array to fit into the 3D matrix
    schedule_reshaped = np.reshape(schedule, (5, 34)).astype(int)  # Convert boolean to integer
    # Assign the reshaped schedule to the corresponding position in the matrix
    availabilities[i] = schedule_reshaped

availabilities.shape
import random 

# Tabu search requires: 
# 1. Initial solution
# 2. Neighborhood function
# 3. Objective function
# 4. Stopping condition
# 5. Tabu list



def do_tabusearch(seed: int):
    # Set random seed
    np.random.seed(seed)
    random.seed(seed)


    # Set availability
    total_participants = 10

    avail_at_t = [[[] for _ in range(34)] for _ in range(5)]
    for day in range(availabilities.shape[1]):
        for time_slot in range(availabilities.shape[2]):
            # print(day, time_slot)
            # Get the availabilities for the current time slot and day
            # free_participants_indices_0_29 = np.where(availabilities[:, day, time_slot][0:total_participants] == 0)[0]
            free_participants_indices_30_39 = np.where(availabilities[:, day, time_slot][30:40] == 0)[0]
            

            
            avail_at_t[day][time_slot] = list(free_participants_indices_30_39)
            
    # print("Final", avail_at_t[1][19])
    avail_at_t

    # Initial solution
    # Randomly select 5 * maxMeetings['maxMeetings'] participants for each timeslot
    participants_per_meeting = 1
    initial_solution = np.zeros((5, 34), dtype=set)
    for day in range(5):
        for time_slot in range(34):
            avail_list =  avail_at_t[day][time_slot]
            num_particip_in_mtg = maxMeetings.iloc[34 * day + time_slot]["Max Meetings"] * participants_per_meeting
            initial_solution[day][time_slot] = sorted(random.sample(avail_list, num_particip_in_mtg))
            
    initial_solution
    # Neighborhood function
    def get_neighborhood(solution):
        neighborhood = []
        for day in range(5):
            for time_slot in range(34):
                for i in range(len(solution[day][time_slot])):
                    # For each element, try +1, +4, -1, -4
                    # Make sure there are no repeats
                    # Validate by making sure all numbers in the new entry are in the avail_at_t list
                    for delta in [1, 4, -1, -4]:
                        # if 0 <= i + delta < len(solution[day][time_slot]):
                        new_participants = solution[day][time_slot].copy()
                        new_participants[i] = (new_participants[i] + delta + total_participants) % total_participants
                        if set(new_participants).issubset(avail_at_t[day][time_slot]) and len(set(new_participants)) == len(solution[day][time_slot]):
                            neighbor = np.copy(solution)
                            neighbor[day][time_slot] = new_participants
                            neighborhood.append(neighbor)

        return neighborhood

    len(get_neighborhood(initial_solution))
    def evaluate_objective(solution):
        # Calculate the number of unique participants in each timeslot
        participant_meetings = np.zeros((total_participants, 5), dtype=int)
        for day in range(5):
            for time_slot in range(34):
                for participant in solution[day][time_slot]:
                    participant_meetings[participant][day] += 1
        
        # find MSE for the same participant across different days
        expected_meetings_between_days = np.mean(participant_meetings, axis=1)

        # Find MSE of number of meetings across different participants
        expected_meetings_between_participants = np.mean(participant_meetings, axis=0)

        # Calculate the penalty score based on the difference between the expected number of meetings and the actual number of meetings
        penalty = 0.0
        for day in range(5):
            delta = np.abs(participant_meetings[:, day] - expected_meetings_between_days)
            penalty += np.sum(delta ** 2)
        
        for participant in range(total_participants):
            delta = np.abs(participant_meetings[participant] - expected_meetings_between_participants)
            penalty += np.sum(delta ** 2)
        
        
        return penalty

    # Stopping condition

    MAX_ITER = 10000
    ITER_NO_CHANGE = 500
    def should_stop(num_iterations: int, equity_history: list[int]):
        if num_iterations >= MAX_ITER:
            return True
        if len(equity_history) >= ITER_NO_CHANGE:
            return len(set(equity_history[-ITER_NO_CHANGE:])) == 1
        return False
        
    # Tabu List
    tabu_list = []
    tabu_list_max_size = 4000

    # Tabu search
    solution = initial_solution.copy()

    best_solution = solution
    best_objective = evaluate_objective(solution)
    equity_history = []

    num_iterations = 0
    while not should_stop(num_iterations, equity_history):
        neighborhood = get_neighborhood(solution)
        best_neighbor = None
        best_neighbor_objective = float('inf')
        for neighbor in neighborhood:
            neighbor_objective = evaluate_objective(neighbor)
            if neighbor_objective < best_neighbor_objective and str(neighbor) not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_objective = neighbor_objective

        if best_neighbor is None:
            break

        solution = best_neighbor
        equity_history.append(best_objective)
        if best_neighbor_objective < best_objective:
            best_solution = best_neighbor
            best_objective = best_neighbor_objective
        
        # if num_iterations % 1000 == 0:
        print(f"Iteration: {num_iterations}, Best Objective: {best_objective}, Best Neighbor Objective: {best_neighbor_objective}")


        tabu_list.append(str(best_neighbor))
        if len(tabu_list) > tabu_list_max_size:
            tabu_list.pop(0)

        num_iterations += 1

    # Save equityHistory to file
    with open('./data/tabusearch/equityHistoryChair.csv', 'at') as f:
        f.write('seed,iteration,equity\n')
        for i, equity in enumerate(equity_history):
            f.write(f'{np.random.get_state()[1][0]},{i},{equity}\n')
    # Save best solution to file
    with open('./data/tabusearch/bestSolutionChair.csv', 'at') as f:
        f.write('seed,day,time,participant\n')
        for day in range(5):
            for time_slot in range(34):
                for participant in best_solution[day][time_slot]:
                    f.write(f'{np.random.get_state()[1][0]},{day},{time_slot},{participant}\n')
    print("Done!!!!")


do_tabusearch(0)
do_tabusearch(423874)
do_tabusearch(2635142)
do_tabusearch(8364142)
do_tabusearch(9473625)



Iteration: 0, Best Objective: 542.6, Best Neighbor Objective: 542.6
Iteration: 1, Best Objective: 514.1999999999999, Best Neighbor Objective: 514.1999999999999
Iteration: 2, Best Objective: 491.0, Best Neighbor Objective: 491.0
Iteration: 3, Best Objective: 467.8, Best Neighbor Objective: 467.8
Iteration: 4, Best Objective: 446.6, Best Neighbor Objective: 446.6
Iteration: 5, Best Objective: 425.4, Best Neighbor Objective: 425.4
Iteration: 6, Best Objective: 404.6, Best Neighbor Objective: 404.6
Iteration: 7, Best Objective: 385.4, Best Neighbor Objective: 385.4
Iteration: 8, Best Objective: 365.40000000000003, Best Neighbor Objective: 365.40000000000003
Iteration: 9, Best Objective: 347.8, Best Neighbor Objective: 347.8
Iteration: 10, Best Objective: 330.59999999999997, Best Neighbor Objective: 330.59999999999997
Iteration: 11, Best Objective: 314.59999999999997, Best Neighbor Objective: 314.59999999999997
Iteration: 12, Best Objective: 299.4, Best Neighbor Objective: 299.4
Iteration: 